# 模型配置

<img src="https://api2.mubu.com/v3/document_image/24054f0e-1156-460a-8146-4c072485ae81-5831216.jpg" width="50%" />

In [1]:
import torch.nn as nn
import torch

In [2]:
"""
数字代表卷积核的个数，‘M’代表最大池化层
"""
cfgs = {
    'vgg11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'vgg19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

In [3]:
"""
提取特征网络结构
"""
def make_features(cfg: list):
    layers = []
    in_channels = 3
    for item in cfg:
        if item == 'M':
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
        else:
            conv2d = nn.Conv2d(in_channels, item, kernel_size=3, padding=1)
            layers.extend([conv2d, nn.ReLU(inplace=True)]) # inplace=True的意思是进行原地操作,节省内存
            in_channels = item
    return nn.Sequential(*layers)

In [4]:
"""
定义整体网络结构
"""
class VGG(nn.Module):
    def __init__(self, features, num_classes=1000, init_weights=False):
        super(VGG, self).__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512*7*7, 2048), # 这里未设置成论文中4096的参数，设置成2048便于演示（减少计算量）
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(2048, 2048),
            nn.ReLU(True),
            nn.Linear(2048, num_classes)
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x): # input: [N, 3, 224, 224]
        x = self.features(x) # output: [N, 512, 7, 7]
        x = torch.flatten(x, start_dim=1) # output: N * 512 * 7 * 7
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
#                 nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
#                 nn.init.normal_(m.weight, 0, 0.01)
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)

In [5]:
"""
定义获取模型实例的函数
"""
def getVGG(model_name='vgg16', ** kwargs):
    try:
        cfg = cfgs[model_name]
    except:
        print(f"Warning: the model-'{model_name}' is not in cfgs dict!")
        exit(-1)
    model = VGG(make_features(cfg), **kwargs)
    return model

In [7]:
model = getVGG('vgg13')
model.parameters

<bound method Module.parameters of VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(256, 512, kernel_size=(3, 3), stride